# keras による CIFAR-10 のデモ

- CIFAR とは直訳するとカナダ先端研究協会 Canadian Institute For Advanced Research 
の提供している画像認識のためのデータセットです。
[CIFAR Home](https://www.cifar.ca/)
- 実際にはトロント大学の [アレックス・クリゼンスキー](https://www.cs.toronto.edu/~kriz/index.html)さんが管理しています。
ちなみにクリゼンスキーさんのファーストネームはアレックスです。
[2012年のイメージネットコンテスト](http://image-net.org/challenges/LSVRC/2012/)でディープラーニングが優勝したときのモデルの名前になった人で，彼の名前からアレックスネットと名付けられました。
- CIFAR10 の説明は [https://www.cs.toronto.edu/~kriz/cifar.html](https://www.cs.toronto.edu/~kriz/cifar.html) を見てください。
簡単に説明すると，10種類(飛行機，自動車，鳥，猫，鹿，犬，蛙，馬，船，トラック)
の対象を認識するためのデータセットで
一枚の画像が縦横 32 画素。全部で50,000枚 + 10,000 枚の画像があります。
- 50,000 枚の画像が訓練画像，10,000 枚がテスト画像です。



In [0]:
/

## 簡単な2層のニューラルネットワークを作ってみましょう

In [0]:
model = Sequential()
model.add(Dense(100, activation='sigmoid', input_dim=3072)) # = 32 x 32 x 3 
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))
model.summary()

In [0]:
from keras.datasets import cifar10

# load CIFAR
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
num_classes = 10

# reshape CIFAR
x_train = x_train.reshape(50000, 32*32*3)
x_test = x_test.reshape(10000, 32*32*3)

# make float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize to (0-1)
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('%d train samples, %d test samples'%(x_train.shape[0], x_test.shape[0]))
print("training data shape: ", x_train.shape, y_train.shape)
print("test data shape: ", x_test.shape, y_test.shape)

In [0]:
samples = np.concatenate([np.concatenate([x_train[i].reshape((32,32,3)) for i in [int(random.random() * len(x_train)) for i in range(16)]], axis=1) for i in range(6)], axis=0)
plt.figure(figsize=(16,6))
plt.imshow(samples, cmap='gray')

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=30,
          validation_data=(x_test, y_test))

## 訓練結果を表示してみます

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# 層を深くしてみます

In [0]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

In [0]:
from keras.datasets import cifar10

# load CIFAR
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
num_classes = 10

# do not reshape CIFAR if you have a convolutional input!

# make float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize to (0-1)
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('%d train samples, %d test samples'%(x_train.shape[0], x_test.shape[0]))
print("training data shape: ", x_train.shape, y_train.shape)
print("test data shape: ", x_test.shape, y_test.shape)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=30,
          validation_data=(x_test, y_test))

## 結果を表示してみます

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
# 画像処理のモジュール
from keras.preprocessing import image

fnames = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]

# 水増しする画像を選択
img_path = fnames[3]

#画像を読み込み, サイズを変更
img = image.load_img(img_path, target_size=(150, 150))

# 形状が(150, 150, 3)のNumPy配列に変換
x = image.img_to_array(img)

#  (1, 150, 150, 3)に変換
x = x.reshape((1,) + x.shape)

# ランダムに変換した画像のバッチを生成する
#無限ループとなってしまうので, 何らかのタイミングでbreakする必要がある
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()
